<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="HEIG-VD Logo" width="100"/>

# Cours TAL - Laboratoire 6
# Trois méthodes de désambiguïsation lexicale

**Objectif**

L'objectif de ce laboratoire est d'implémenter et de comparer plusieurs méthodes de désambiguïsation lexicale (en anglais, *Word Sense Disambiguation* ou WSD).  Vous utiliserez un corpus avec plusieurs milliers de phrases, chaque phrase contenant une occurrence du mot anglais *interest* annotée avec le sens que ce mot possède dans la phrase respective.  Les trois méthodes sont les suivantes (elles seront détaillées par la suite) :

* Algorithme de Lesk simplifié.
* Utilisation de word2vec.
* Classification supervisée utilisant des traits lexicaux.

Les deux premières méthodes n'utilisent pas l'apprentissage automatique.  Elles fonctionnent selon le même principe : comparer le contexte d'une occurrence de *interest* avec chacune des définitions des sens (*synsets*) et choisir la définition la plus proche du contexte.  L'algorithme de Lesk définit la proximité comme le nombre de mots en commun, alors que word2vec la calcule comme la similarité de vecteurs.  La dernière méthode vise à classifier les occurrences de *interest*, les sens étant les classes, et les attributs étant les mots du contexte (apprentissage supervisé).

## 1. Analyse des données

Téléchargez le corpus *interest* depuis le [site du Prof. Ted Pedersen](http://www.d.umn.edu/~tpederse/data.html) (il se trouve en bas de sa page web).  Téléchargez l'archive ZIP marquée *original format without POS tags* et extrayez le fichier `interest-original.txt`.  Téléchargez également le fichier `README.int.txt` indiqué à la ligne au-dessus. Veuillez répondre brièvement aux questions suivantes :

a. Quelles sont les URL du fichier ZIP et celle du fichier `README.int.txt` ?

b. Quel est le format du fichier `interest-original.txt` et comment sont annotés les sens de *interest* ?

c. Est-ce qu'il y a aussi des occurrences au pluriel (*interests*) à traite ?

d. Comment sont annotées les phrases qui contiennent plusieurs occurrences du mot *interest* ?

In [1]:
# Veuillez répondre ici (en commentaire) aux questions.

# a. Les URL sont les suivantes :
# https://www.d.umn.edu/~tpederse/Data/interest-original.nopos.tar.gz
# https://www.d.umn.edu/~tpederse/Data/README.int.txt

# b. Le format du fichier `interest-original.txt` est le suivant : chaque phrase est délimitée par des $$.
# Aussi, chaque occurrence du mot *interest* est écrite avec le numéro de son sens. Par exemple, interest_6.

# c. Oui, il y a aussi des occurrences au pluriel. Elles sont annotées de la même manière que les occurrences au singulier,
# mais avec un "s" à la fin. Par exemple, interests_4.

# d. Les phrases contienant plusieurs occurrences du mot *interest* ne sont annotées qu'une seule fois.

# Par exemple, avec la phrase "it  is n't *interested in mounting  ...  the growing interest_1  in  the british concern"
# on voit que le premier mot "interested" commence par un * qui indique qu'il ne s'agit pas de l'occurence du mot *interest*
# qui nous intéresse. Il faut donc ignorer ce mot. De plus, le mot "interest_1" est annoté avec le numéro de son sens, ici 1.

# La phrase est ensuite dupliquée en échangant l'intérêt pour les deux mots *interest*.
# Celui qui était ignoré devient celui qui est annoté et l'autre est ignoré avec un *.

**1e.** D'après le fichier `README.int.txt`, quelles sont les définitions des six sens de *interest* annotés dans les données et quelles sont leurs fréquences ? Vous pouvez copier/coller l'extrait de `README`ici.

In [2]:
# Veuillez répondre ici (en commentaire) à la question.

# Sense 1 =  361 occurrences (15%) - readiness to give attention (En français : disposition à prêter attention)
# Sense 2 =   11 occurrences (01%) - quality of causing attention to be given to (En français : qualité de provoquer l'attention)
# Sense 3 =   66 occurrences (03%) - activity, etc. that one gives attention to (En français : activité, etc. à laquelle on prête attention)
# Sense 4 =  178 occurrences (08%) - advantage, advancement or favor (En français : avantage, avancement ou faveur)
# Sense 5 =  500 occurrences (21%) - a share in a company or business (En français : une part dans une société ou une entreprise)
# Sense 6 = 1252 occurrences (53%) - money paid for the use of money (En français : argent payé pour l'utilisation de l'argent)


**1f.** De quel dictionnaire viennent les sens précédents ? Où peut-on le consulter en ligne ?  Veuillez aligner les définitions du dictionnaire avec les six sens annotés en écrivant par exemple `Sense 3 = "an activity that you enjoy doing or a subject that you enjoy studying"`.

In [3]:
# Veuillez répondre ici (en commentaire) à la question.

# Les senses viennent de la version électronique de la première édition du Longman's Dictionary of Contemporary English.

# Nous pouvons le consulter en ligne à l'adresse suivante : https://www.ldoceonline.com/

# Sens 1 = "if you have an interest in something or someone, you want to know or learn more about them"
# Sens 2 = "a quality or feature of something that attracts your attention or makes you want to know more about it"
# Sense 3 = "an activity that you enjoy doing or a subject that you enjoy studying"
# Sens 4 = "the things that bring advantages to someone or something"
# Sens 5 = "if you have an interest in a particular company or industry, you own shares in it"
# Sens 6 = "the extra money that you must pay back when you borrow money"


**1g.** En consultant [WordNet en ligne](http://wordnetweb.princeton.edu/perl/webwn), trouvez les définitions des synsets  pour le **nom commun** *interest*.  Combien de synsets y a-t-il ?  Veuillez indiquer comme avant la **définition** de chaque synset pour chacun des six sens ci-dessus (au besoin, fusionner ou ignorer des synsets).

In [4]:
# Veuillez répondre ici (en commentaire) à la question.

# Sens 1 = "a sense of concern with and curiosity about someone or something"
# Sens 2 = "the power of attracting or holding one's attention "
# Sense 3 = "a social group whose members control some field of activity and who have common aims"
# Sens 4 = "a reason for wanting something done in the common interest"
# Sens 5 = "a right or legal share of something; a financial involvement with something"
# Sens 6 = "a fixed charge for borrowing money; usually a percentage of the amount borrowed"

**1h.** Définissez (manuellement, ou avec quelques lignes de code) une liste nommée `senses1` avec les mots des définitions du README, en supprimant les stopwords (p.ex. les mots < 4 lettres).  Affichez la liste.

In [5]:
import nltk

In [6]:
from random import randrange

In [7]:
# Veuillez répondre ici à la question et créer la variable 'senses1' (liste de 6 listes de chaînes).
senses1 = []

readme_string = "readiness to give attention\nquality of causing attention to be given to\nactivity etc that one gives attention to\nadvantage advancement or favor\na share in a company or business\nmoney paid for the use of money"


for sentence in readme_string.split("\n"):
    tokens = sentence.split(" ")
    senses = set([word for word in tokens if len(word) > 3])
    senses1.append(senses)

print(senses1)
print(len(senses1))
print(sum([len(sense) for sense in senses1]))

[{'readiness', 'give', 'attention'}, {'causing', 'attention', 'given', 'quality'}, {'attention', 'that', 'activity', 'gives'}, {'advancement', 'advantage', 'favor'}, {'company', 'business', 'share'}, {'paid', 'money'}]
6
19


**1i.** En combinant les définitions obtenues aux points (4) et (5) ci-dessus, construisez une liste nommée `senses2` avec pour chacun des sens de *interest* une liste de **mots-clés** correspondants.  Vous pouvez concaténer les définitions, puis écrire des instructions en Python pour extraire les mots (uniques).  Respectez l'ordre des sens données par `README`, et à la fin affichez `senses2`.

In [9]:
# Veuillez répondre ici à la question et créer la variable 'senses2' (liste de 6 listes de chaînes).
senses2 = []

string_f = "if you have an interest in something or someone, you want to know or learn more about them\na quality or feature of something that attracts your attention or makes you want to know more about it\nan activity that you enjoy doing or a subject that you enjoy studying\nthe things that bring advantages to someone or something\nif you have an interest in a particular company or industry, you own shares in it\nthe extra money that you must pay back when you borrow money"

string_g = "a sense of concern with and curiosity about someone or something\nthe power of attracting or holding one's attention\na social group whose members control some field of activity and who have common aims\na reason for wanting something done in the common interest\na right or legal share of something; a financial involvement with something\na fixed charge for borrowing money; usually a percentage of the amount borrowed"

string_f_split = string_f.split("\n")
string_g_split = string_g.split("\n")

for i in range(len(string_f_split)):
    tokens_f = string_f_split[i].split(" ")
    tokens_g = string_g_split[i].split(" ")
    tokens = tokens_f + tokens_g
    senses = set([word for word in tokens if len(word) > 3])
    senses2.append(senses)

print(senses2)
print(len(senses2))
print(sum([len(sense) for sense in senses2]))

[{'with', 'want', 'learn', 'them', 'concern', 'someone,', 'know', 'sense', 'interest', 'about', 'something', 'curiosity', 'have', 'more', 'someone'}, {'makes', 'want', 'your', 'that', 'attracts', 'know', 'power', 'attracting', "one's", 'about', 'something', 'quality', 'more', 'attention', 'feature', 'holding'}, {'whose', 'activity', 'subject', 'social', 'group', 'common', 'enjoy', 'that', 'studying', 'doing', 'field', 'aims', 'have', 'members', 'some', 'control'}, {'reason', 'things', 'wanting', 'common', 'that', 'bring', 'interest', 'done', 'something', 'advantages', 'someone'}, {'with', 'legal', 'involvement', 'share', 'right', 'company', 'something;', 'interest', 'particular', 'financial', 'something', 'have', 'shares', 'industry,'}, {'fixed', 'borrow', 'borrowed', 'that', 'must', 'when', 'money;', 'amount', 'usually', 'back', 'borrowing', 'extra', 'money', 'charge', 'percentage'}]
6
87


**1j.** Chargez les données depuis `interest-original.txt` dans une liste appelée `sentences` qui contient pour chaque phrase la liste des mots (sans les séparateurs *$$* et *===...*).  Ces phrases sont-elles déjà tokenisées en mots ?  Sinon, faites-le.  À ce stade, ne modifiez pas encore les occurrences annotées *interest(s)\_X*.  Comptez le nombre total de phrases et affichez-en trois au hasard.

In [13]:
import re

# Veuillez répondre ici à la question.
sentences = []

with open("data/interest-original.txt", "r", encoding="utf-8") as f:
    text = f.read()

# Séparer les phrases sur "$$"
raw_sentences = text.split("$$")

sentences = []
for raw in raw_sentences:
    # Nettoyer les sauts de ligne et espaces inutiles
    cleaned = raw.strip()
    if not cleaned:
        continue

    # Supprimer toute séquence "====...====", peu importe où elle apparaît
    cleaned = re.sub(r"={5,}.*?={5,}", "", cleaned)

    # Re-nettoyer les espaces et tokeniser
    tokens = cleaned.strip().split()
    if tokens:
        sentences.append(tokens)


print("Il y a {} phrases.\nEn voici 3 au hasard :".format(len(sentences)))
print(sentences[151:154])

Il y a 2368 phrases.
En voici 3 au hasard :
[['investor', 'interest_1', 'in', 'stock', 'funds', '``', 'has', "n't", 'stalled', 'at', 'all', ',', "''", 'mr.', 'hines', 'maintains', '.'], ["''", 'it', 'is', 'in', 'the', 'western', 'interest_4', 'to', 'see', 'mr.', 'gorbachev', 'succeed', '.'], ['revco', 'insists', 'that', 'the', 'proposal', 'is', 'simply', 'an', '``', 'expression', 'of', 'interest_1', ',', "''", 'because', 'under', 'chapter', '11', 'revco', 'has', '``', 'exclusivity', 'rights', "''", 'until', 'feb.', '28', '.']]


## 2. Algorithme de Lesk simplifié

**2a.** Définissez une fonction `wsd_lesk(senses, sentence)` qui prend deux arguments : une liste de listes de mots-clés (comme `senses1` et `senses2` ci-dessus) et une phrase avec une occurrence annotée de *interest* ou *interests*, et qui retourne l'index du sens le plus probable (entre 1 et 6) selon l'algorithme de Lesk.  Cet algorithme choisit le sens qui a le maximum de mots en commun avec le contexte de *interest*.  Vous pouvez choisir vous-mêmes la taille de ce voisinage (`window_size`).  En cas d'égalité entre deux sens, tirer la réponse au sort.

In [ ]:
# Veuillez répondre ici à la question.


**2b.** Définissez maintenant une fonction `evaluate_wsd(fct_name, senses, sentences)` qui prend en paramètre le nom de la méthode de similarité (pour commencer : `wsd_lesk`) ainsi que la liste des mots-clés par sens, et la liste de phrases, et qui retourne le score de la méthode de similarité.  Ce score sera tout simplement le pourcentage de réponses correctes (sens trouvé identique au sens annoté).

In [ ]:
# Veuillez répondre ici à la question.


**2c.** En fixant au mieux la taille de la fenêtre autour de *interest*, quel est le meilleur score de la méthode de Lesk simplifiée ?  Quelle liste de sens conduit à de meilleurs scores, `senses1` ou `senses2` ?

In [ ]:
# Veuillez répondre ici à la question.


## 3. Utilisation de word2vec pour la similarité contexte vs. synset

**3a.** En réutilisant une partie du code de `wsd_lesk`, veuillez maintenant définir une fonction `wsd_word2vec(senses, sentence)` qui choisit le sens en utilisant la similarité **word2vec** étudiée dans le labo précédent. 
* Vous pouvez chercher dans la [documentation des KeyedVectors](https://radimrehurek.com/gensim/models/keyedvectors.html) comment calculer directement la similarité entre deux listes de mots.
* Comme `wsd_lesk`, la nouvelle fonction `wsd_word2vec` prend en argument une liste de listes de mots-clés par sens (comme `senses1` et `senses2` ci-dessus), et une phrase avec une occurrence annotée de *interest* ou *interests*.
* La fonction retourne le numéro du sens le plus probable selon la similarité word2vec entre les mots du sens et ceux du voisinage de *interest*.  En cas d'égalité, tirer le sens au sort.
* Vous pouvez régler la taille du voisinage (`window_size`) par l'expérimentation.  

In [ ]:
import gensim
from gensim.models import KeyedVectors
path_to_model = "../../../gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz" # à adapter
wv_model = gensim.models.KeyedVectors.load_word2vec_format(path_to_model, binary=True)  # C bin format

In [ ]:
# Veuillez répondre ici à la question.


**3b.** Appliquez maintenant la même méthode `evaluate_wsd` avec la fonction `wsd_word2vec` (en cherchant une bonne valeur de la taille de la fenêtre) et affichez le score de la similarité word2vec.  Comment se compare-t-il avec le score précédent (Lesk) ?

In [ ]:
# Veuillez répondre ici à la question.


## 4. Classification supervisée avec des traits lexicaux
Vous entraînerez maintenant des classifieurs pour prédire le sens d'une occurrence dans une phrase.  Le premier but sera de transformer chaque phrase en un ensemble d'attributs pour formater les données en vue des expériences de classification.

Veuillez utiliser le classifieur `NaiveBayesClassifier` fourni par NLTK.  Le mode d'emploi se trouve dans le [Chapitre 6, sections 1.1-1.3](https://www.nltk.org/book/ch06.html) du livre NLTK.  Consultez-le attentivement pour trouver comment formater les données.  De plus, il faudra séparer les données en sous-ensembles d'entraînement et de test.

On vous propose de nommer les attributs `word-k`, ..., `word-2`, `word-1`, `word+1`, `word+2`, ..., `word+k` (fenêtre de taille `2*k` autour de *interest*).  Leurs valeurs sont les mots observés aux emplacements respectifs, ou `NONE` si la position dépasse l'étendue de la phrase.  Vous ajouterez un attribut nommé `word0` qui est l'occurrence du mot *interest* au singulier ou au pluriel.  

Pour chaque occurrence de *interest*, vous devrez donc créer la représentation suivante (où `6` est le numéro du sens, essentiel pour l'entraînement, mais à cacher lors de l'évaluation) :
```
[{'word-1': 'in', 'word+1': 'rates', 'word-2': 'declines', 'word+2': 'NONE', 'word0': 'interest'}, 6]
```

**4a.** En partant de la liste des phrases appelée `sentences` préparée plus haut, veuillez générer la liste avec toutes les représentation, appelée `items_with_features`.  Vous pouvez vous aider du livre NLTK.

In [ ]:
# Veuillez répondre ici à la question.

print(len(items_with_features))
print(items_with_features[151:154])

**4b.** Veuillez séparer les données aléatoirement en 80% pour l'entraînement et 20%  pour l'évaluation.  Veuillez faire une division stratifiée : les deux sous-ensembles doivent contenir les mêmes proportions de sens que l'ensemble de départ.  Ils seront appelés `iwf_train` et `iwf_test`.

In [ ]:
from random import shuffle

In [ ]:
iwf_train = []
iwf_test  = []
# Veuillez répondre ici à la question.

print(len(iwf_train), ' ', len(iwf_test))
print(iwf_test[:2], iwf_test[-2:])

**4c.** Veuillez créer une instance de `NaiveBayesClassifier`, l'entraîner sur `iwf_train` et la tester sur `iwf_test` (voir la documentation NLTK).  En expérimentant avec différentes largeurs de fenêtres, quel est le meilleur score que vous obtenez (avec la fonction `accuracy` de NLTK) sur l'ensemble de test ?  Comment se compare-t-il avec les précédents ?

In [ ]:
from nltk.classify import naivebayes 
# Veuillez répondre ici à la question.


**4d.** En utilisant la fonction `show_most_informative_features()`, veuillez afficher les attributs les plus informatifs et commenter le résultat.

**4e.** On souhaite également obtenir les scores pour chaque sens.  Pour ce faire, il faut demander les prédictions une par une au classifieur (voir le [livre NLTK](https://www.nltk.org/book/ch06.html)), et comptabiliser les prédictions correctes pour chaque sens.  Vous pouvez vous inspirer de `evaluate_wsd`, et écrire une fonction `evaluate_wsd_supervised(classifier, items_with_features)`, que vous appliquerez aux donnés `iwf_test`.  Veuillez afficher ces scores.

In [ ]:
def evaluate_wsd_supervised(classifier, items_with_features):
# à compléter

evaluate_wsd_supervised(classifier, iwf_test)

## 5. Conclusion

Veuillez recopier ci-dessous, en guise de conclusion, les scores des trois expériences réalisées, pour pouvoir les comparer d'un coup d'oeil.  Quel est le meilleur score obtenu?

## Fin du laboratoire

Merci de nettoyer votre feuille, sauvegarder le résultat, et soumettre le *notebook* sur Cyberlearn.